In [1]:
import os
import random
from os.path import join
from time import time

import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from torch import optim
from torch.nn import BCELoss
from torch.optim import Adam
from torch.utils.data import TensorDataset
from torchvision.utils import save_image

from models.de_PAMAP2 import DAE
from models.dis_z import DIS_Z
from models.activity_recognition import *
from utils.function import *

# from matplotlib import pyplot as plt
# %pylab inline

In [3]:
class Args:
    def __init__(self):
        self.root = '../data'
        self.batchSize = 64
        self.maxEpochs = 100
        self.nz = 200
        self.lr = 1e-4
        self.fSize = 64
        self.outDir = 'data/experiments/DE_EVAL_PAMAP_Missing'
        self.commit = 'eval'
        self.alpha = 1.0
        # self.sigma = 0.35
        self.M = 5
        self.loss = 'MSE' #'BCE'
        self.loadDAE = False
        self.loadSVM = False    
        self.load_DAE_from = None
        self.evalMode = False
        self.comment = ''
        self.momentum = 0.1
        self.c = 0.01
        self.svmLR = 1e-4
        self.Ntest = 100
        self.gpuNo = 0
        self.multimodalZ = False
        self.window_len = 512
        self.stride_len = 20
        self.act_list = [1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24]
        self.imSize = 64
        self.sigma = [40, 80]
        # self.sigma = 0.1
        # self.sigma = [0.1, 40, 80]
        self.cuda_id = 0
        self.random_seed = 2
        self.train_split = 0.8

In [4]:
def build_dis(args, dae, multimodalZ):
    if not multimodalZ:
        print('\n ** USING NORMAL PRIOR **')
        prior = dae.norm_prior
        NZ = args.nz
    else:
        print('\n ** USING MULTIMODAL PRIOR **')
        prior = dae.multi_prior
        NZ = 2
    dis = DIS_Z(nz=NZ, prior=prior)

    return dis, NZ

In [7]:
def train_dae(args, exDir, trainLoader, testLoader):
    dae = DAE(nz=args.nz, imSize=args.imSize, fSize=args.fSize, sigma=args.sigma, multimodalZ=args.multimodalZ) #sigma=level of corruption
    # dis, NZ = build_dis(args, dae=dae, multimodalZ=args.multimodalZ)
    
    if dae.useCUDA:
        torch.cuda.set_device(args.gpuNo)
        dae.cuda()
        # dis.cuda()
    
    save_input_args(exDir, args)  #save training opts
    save_exp_details(args, exDir)
    
    # #Create optimizers
    optimDAE = optim.RMSprop(dae.parameters(), lr = args.lr, momentum=args.momentum)
    # optimDIS = optim.RMSprop(dis.parameters(), lr = args.lr, momentum=args.momentum)

    # #Keeping track of training
    losses = {'enc': [], 'rec': [], 'dis':[], 'test rec':[]}

    # xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)

    with torch.autograd.set_detect_anomaly(True):
        for e in range(args.maxEpochs):

            epochEncLoss=0
            epochRecLoss=0
            epochDisLoss=0

            for i, data in enumerate(trainLoader):
                # print(i)

                T = time()

                dae.train()
                # dis.train()

                x, y = prep_data(data, useCUDA=dae.useCUDA)
                # print(x.shape)
            
                # get outputs
                zFake, xRec = dae.forward(x)

                # clac losses
                recLoss = dae.rec_loss(xRec, x, loss=args.loss)  #loss='BCE' or 'MSE'
            
                optimDAE.zero_grad()
                recLoss.backward()
                optimDAE.step()            
            
                # zFake, xRec = dae.forward(x)    
            
                # disLoss = dis.dis_loss(zFake)
                #do updates

                # optimDIS.zero_grad()
                # disLoss.backward()
                # optimDIS.step()
            
#                 encLoss = dis.gen_loss(zFake)
            
#                 optimDAE.zero_grad()
#                 encLoss.backward()
#                 optimDAE.step()                   


                # epochEncLoss+=encLoss.detach().cpu().numpy()
                epochRecLoss+=recLoss.detach().cpu().numpy()
                # epochDisLoss+=disLoss.detach().cpu().numpy()
            
                if i%100 == 0:
                    print('[%d, %d] rec: %0.5f, time: %0.3f' % (e, i, recLoss.detach().cpu().numpy(), time() - T))
                    # print('[%d, %d] enc: %0.5f, rec: %0.5f, dis: %0.5f, time: %0.3f' % (e, i, encLoss.detach().cpu().numpy(), recLoss.detach().cpu().numpy(), disLoss.detach().cpu().numpy(), time() - T))

            # storing losses for plotting later
            # losses['enc'].append(epochEncLoss/i)
            losses['rec'].append(epochRecLoss/i)
            # losses['dis'].append(epochDisLoss/i)

            #### Test
            dae.eval()
            # dis.eval()

            # #get test outuputs and losses
            # xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)
            # zTest, recTest = dae.forward(xTest)  #N.B. corruption in here
            # recLossTest = dae.rec_loss(recTest, xTest)
            # #Plot losses
            # # losses['test rec'].append(recLossTest.data[0])
            # losses['test rec'].append(recLossTest.detach().cpu().numpy())

#             if e > 0: #only one point for test rec otherwise
#                 plot_losses(losses, exDir, epochs=e+1)
#                 plot_norm_losses(losses, exDir)

            #save parameters
            dae.save_params(exDir)
            # dis.save_params(exDir)

            #Save images of original and rec
            # save_image(xTest.data, join(exDir, 'original.png'))
            # save_image(recTest.data, join(exDir, 'rec.png'))

            #Save samples
            # sampleDir = join(exDir,'epoch_'+str(e))
            # os.mkdir(sampleDir)
            # print('sample dir:', sampleDir)
            # dae.sample_x(args.M, sampleDir)

In [8]:
# if __name__ == "__main__":
args = Args()

random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)

# X, labels = prepare_data(args)
# trainLoader, testLoader = prepare_dataloaders(args, X, labels)

X_train, X_test, y_train, y_test = prepare_data_PAMAP2(args)

# tail = len(X_test) % args.batchSize
# X = torch.from_numpy(X_test[:len(X_test)-tail,:,:])
# labels = torch.from_numpy(y_test[:len(X_test)-tail,:])
trainDataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
testDataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
trainLoader = torch.utils.data.DataLoader(trainDataset,
    batch_size=args.batchSize, shuffle=True, drop_last=True) 

testLoader = torch.utils.data.DataLoader(testDataset,
    batch_size=args.batchSize, shuffle=False, drop_last=True)

exDir = make_new_folder(args.outDir)
print('Outputs will be saved to:', exDir)


#train_activity_recognition(args, exDir, trainLoader, testLoader)

Outputs will be saved to: data/experiments/DE_EVAL_PAMAP_Missing/Ex_7


In [9]:
train_dae(args, exDir, trainLoader, testLoader)


Experimental details:
root = ../data
batchSize = 64
maxEpochs = 100
nz = 200
lr = 0.0001
fSize = 64
outDir = data/experiments/DE_EVAL_PAMAP_Missing
commit = eval
alpha = 1.0
M = 5
loss = MSE
loadDAE = False
loadSVM = False
load_DAE_from = None
evalMode = False
comment = 
momentum = 0.1
c = 0.01
svmLR = 0.0001
Ntest = 100
gpuNo = 0
multimodalZ = False
window_len = 512
stride_len = 20
act_list = [1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24]
imSize = 64
sigma = [40, 80]
cuda_id = 0
random_seed = 2
train_split = 0.8


/home/xyang18/miniconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


[0, 0] rec: 0.01695, time: 0.619
[0, 100] rec: 0.00480, time: 0.121
[0, 200] rec: 0.00459, time: 0.109
[0, 300] rec: 0.00460, time: 0.118
[0, 400] rec: 0.00546, time: 0.119
[0, 500] rec: 0.00448, time: 0.119
[0, 600] rec: 0.00446, time: 0.109
[0, 700] rec: 0.00411, time: 0.124
[0, 800] rec: 0.00352, time: 0.097
[0, 900] rec: 0.00430, time: 0.125
[0, 1000] rec: 0.00364, time: 0.114
[0, 1100] rec: 0.00337, time: 0.109
saving params...
[1, 0] rec: 0.00359, time: 0.175
[1, 100] rec: 0.00375, time: 0.135
[1, 200] rec: 0.00290, time: 0.146
[1, 300] rec: 0.00328, time: 0.117
[1, 400] rec: 0.00277, time: 0.117
[1, 500] rec: 0.00257, time: 0.123
[1, 600] rec: 0.00290, time: 0.091
[1, 700] rec: 0.00255, time: 0.115
[1, 800] rec: 0.00239, time: 0.119
[1, 900] rec: 0.00233, time: 0.095
[1, 1000] rec: 0.00228, time: 0.118
[1, 1100] rec: 0.00246, time: 0.128
saving params...
[2, 0] rec: 0.00223, time: 0.211
[2, 100] rec: 0.00246, time: 0.116
[2, 200] rec: 0.00244, time: 0.120
[2, 300] rec: 0.00207, 